In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
client = bigquery.Client()
query = """
SELECT *
FROM (
SELECT *, RANK() OVER (PARTITION BY SEARCH_WEEK ORDER BY WKLY_VIEWS DESC) AS WKLY_VIEWS_RANK
FROM (
SELECT WIK.title, EXTRACT(WEEK FROM datehour) AS SEARCH_WEEK, SUM(WIK.views) AS WKLY_VIEWS
FROM `bigquery-public-data.wikipedia.pageviews_2023` WIK
WHERE DATE(datehour) >= "2023-01-01"
AND TRIM(UPPER(WIK.wiki)) = 'EN.M'
AND TRIM(UPPER(WIK.title)) NOT IN ('MAIN_PAGE','SPECIAL:SEARCH')
AND UPPER(WIK.title) NOT LIKE '%WIKIPEDIA%'
GROUP BY 1,2))
WHERE WKLY_VIEWS_RANK <= 10
ORDER BY 2, 3 DESC;
"""
df = pd.read_gbq(query)
df.to_csv("C:/Users/jakes/Documents/WikipediaTop2023SearchTermsByWeek.csv", index=False)